# Import Dependancies


In [ ]:
import nltk
import json
import numpy as np
nltk.download('punkt')  # uncomment this if it's not downloaded, comment it out again after downloading
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

#Define Hyperaramters

In [ ]:
BATCH_SIZE = 8
WORKERS = 2
HIDDEN_SIZE = 8
LEARNING_RATE = 0.001
EPOCHS = 800

#Process Input Data

In [ ]:
def tokenize(text):
    return nltk.word_tokenize(text)

def stem(word):
    # Lowercases the word and stemming
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_words, all_words):
    """
    tokenized_words: ["hello", how", "are", "you"]
    all_words: ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    returns: [0, 1, 0, 1, 0, 0, 0]
    """
    tokenized_words = [stem(w) for w in tokenized_words]
    bag = np.zeros(len(all_words), dtype=np.float32) #[0, 0, 0, 0, 0, 0, 0]
    for idx, w in enumerate(all_words):
        if w in tokenized_words:
            bag[idx] = 1.0        #Flips the them to 1
    return bag

def get_json_data():
    with open("intents.json", "r") as f:
        data = json.load(f)


    all_words = []
    tags = []
    xy = []
    for intent in data["intents"]:
        tag = intent["tag"]
        tags.append(tag)

        # 1. Tokenization
        for pattern in intent["patterns"]:
            w = tokenize(pattern)
            all_words.extend(w) # extend adds all the elements of w to all_words
            xy.append((w, tag))

    ignore_words = ["?", "!", ".", ","]


    #2. Lowercasing (done in stemming function) and 3. Stemming and
    all_words = [stem(w) for w in all_words if w not in ignore_words] #4. Remove Punctuation
    all_words = sorted(set(all_words))
    tags = sorted(set(tags))

    # print(len(xy), "patterns")
    # print(len(tags), "tags:", tags)
    # print(len(all_words), "unique stemmed words:", all_words)

    return all_words, tags, xy, data

#Neural Network Class


In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        return out

#Dataset Class

In [ ]:
class ChatDataset(Dataset):
    def __init__(self, x, y):
        self.n_samples = len(x)
        self.x_data = x
        self.y_data = y

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples

#Get Training Data

In [ ]:
def get_training_data():
    all_words, tags, xy, intents = get_json_data()

    X_train = []
    Y_train = []

    for (pattern, tag) in xy:
        bag = bag_of_words(pattern, all_words)
        X_train.append(bag)

        label = tags.index(tag)
        Y_train.append(label)

    X_train = np.array(X_train)
    Y_train = np.array(Y_train)

    return X_train, Y_train, tags

#Training Loop

In [ ]:
def train(model):
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
  for epoch in range(EPOCHS):
      for (words, labels) in train_loader:
          words = words.to(device)
          labels = labels.to(dtype=torch.long).to(device)

          #forward pass
          outputs = model(words)
          loss = criterion(outputs, labels)

          #backward and optimizer step
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

      if (epoch + 1) % 100 == 0:
          print(f'epoch {epoch + 1}/{EPOCHS}, loss={loss.item():.3f}')

  print(f'Final Loss, loss={loss.item():.3f}')


#Model Set Up

In [ ]:
X_train, Y_train, tags = get_training_data()

dataset = ChatDataset(x=X_train, y=Y_train)
train_loader = DataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=WORKERS)

model = NeuralNet(input_size=len(X_train[0]), hidden_size=HIDDEN_SIZE, num_classes=len(tags))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


NeuralNet(
  (l1): Linear(in_features=81, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=8, bias=True)
  (relu): ReLU()
)

In [ ]:
train(model)

epoch 100/800, loss=0.361
epoch 200/800, loss=0.316
epoch 300/800, loss=0.002
epoch 400/800, loss=0.003
epoch 500/800, loss=0.010
epoch 600/800, loss=0.001
epoch 700/800, loss=0.001
epoch 800/800, loss=0.000
Final Loss, loss=0.000


#Save the Model

In [ ]:
all_words, tags, xy, intents = get_json_data()

data = {
    "model_state": model.state_dict(),
    "input_size": len(X_train[0]),
    "hidden_size": HIDDEN_SIZE,
    "output_size": len(tags),
    "all_words": all_words,
    "tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)


#Load and Run Model

In [ ]:
def load_model(file_name):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  with open("intents.json", "r") as f:
      intents = json.load(f)

  data = torch.load(file_name)

  input_size = data["input_size"]
  hidden_size = data["hidden_size"]
  output_size = data["output_size"]
  all_words = data["all_words"]
  tags = data["tags"]
  model_state = data["model_state"]

  model = NeuralNet(input_size, hidden_size, output_size).to(device)
  model.load_state_dict(model_state)
  model.eval()
  return model, all_words, tags, intents

In [ ]:
bot_name = "Axel"
print(f"Hi I'm {bot_name} Let's chat! (type 'quit' to exit)")
chat_model, all_words, tags, intents = load_model("data.pth")

Let's Chat


In [ ]:
import random

In [ ]:
def get_response(model, all_words, tags, sentence, intents):
    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)

    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                return random.choice(intent['responses'])

    return "I do not understand. Can you please try another question or rephrase your question"

In [ ]:
while True:
  sentence = input()
  if sentence == "quit":
    break

  print(f"{bot_name} : {get_response(model, all_words, tags, sentence, intents)}")

hi
Axel : Hi there, how can I help?
vhnvnhn
Axel : Hi there, what can I do for you?
tell me a joke
Axel : Why don't scientists trust atoms? Because they make up everything!
how do i pay
Axel : We accept VISA, Mastercard, American Express and Paypal
can i pay with cash
Axel : We accept most major credit cards, and Paypal
quit
